## 5 模型验证

## 5.3.1 模型过拟合与欠拟合¶

In [3]:
# 导入工具
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

import warnings
warnings.filterwarnings("ignore")

from sklearn.linear_model import LinearRegression  #线性回归
from sklearn.neighbors import KNeighborsRegressor  #K近邻回归
from sklearn.tree import DecisionTreeRegressor     #决策树回归
from sklearn.ensemble import RandomForestRegressor #随机森林回归
from sklearn.svm import SVR  #支持向量回归
import lightgbm as lgb #lightGbm模型

from sklearn.model_selection import train_test_split # 切分数据
from sklearn.metrics import mean_squared_error #评价指标

from sklearn.linear_model import SGDRegressor

In [4]:
train_data_file = "./zhengqi_train.txt"
test_data_file = "./zhengqi_test.txt"

train_data = pd.read_csv(train_data_file, sep="\t",encoding="utf-8")
test_data = pd.read_csv(test_data_file, sep="\t", encoding="utf-8")

In [9]:
# 归一化
from sklearn import preprocessing 

#['v0', ... 'v37']

features_columns = [col for col in train_data.columns if col not in ['target']]

min_max_scaler = preprocessing.MinMaxScaler()

min_max_scaler.fit(train_data[features_columns])

train_data_by_scaler = min_max_scaler.transform(train_data[features_columns])
test_data_by_scaler = min_max_scaler.transform(test_data[features_columns])

train_data_scaler = pd.DataFrame(train_data_by_scaler)
train_data_scaler.columns = features_columns

test_data_scaler = pd.DataFrame(test_data_by_scaler)
test_data_scaler.columns = features_columns

train_data_scaler['target'] = train_data['target']

In [12]:
train_data_scaler

,V0,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V29,V30,V31,V32,V33,V34,V35,V36,V37,target
0,0.759139,0.729830,0.524488,0.680337,0.698964,0.479596,0.427136,0.385874,0.613522,0.757557,...,0.406834,0.641467,0.666159,0.679280,0.000000,0.000000,0.074074,0.000000,0.018401,0.175
1,0.821406,0.789631,0.557939,0.705130,0.664244,0.482591,0.465152,0.385874,0.724232,0.757557,...,0.371596,0.643552,0.748349,0.721619,0.374950,0.499949,0.755580,0.289702,0.437406,0.676
2,0.828377,0.808239,0.584987,0.674567,0.653210,0.518532,0.495905,0.385874,0.733458,0.757557,...,0.387480,0.676487,0.779472,0.610577,0.374950,0.499949,0.755580,0.429901,0.458673,0.633
3,0.785006,0.779830,0.592670,0.642601,0.718746,0.562711,0.521712,0.425208,0.734467,0.757557,...,0.390683,0.684269,0.779726,0.722084,0.374950,0.477220,0.755580,0.374841,0.530618,0.206
4,0.777416,0.818182,0.588988,0.649462,0.683488,0.646949,0.541338,0.425208,0.721638,0.757557,...,0.413107,0.776126,0.785950,0.693393,0.374950,0.462067,0.755580,0.296712,0.543288,0.384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2883,0.700898,0.724006,0.525288,0.641977,0.718880,0.737551,0.824138,0.808498,0.672625,0.969703,...,0.405766,0.597415,0.847053,0.601737,0.482957,0.481059,0.666667,0.405812,0.650980,0.235
2884,0.750000,0.806676,0.594750,0.645408,0.709460,0.728192,0.822902,0.808498,0.753784,0.969703,...,0.427523,0.586437,0.803354,0.580490,0.534086,0.534094,0.666667,0.254015,0.490196,1.042
2885,0.610440,0.625142,0.469750,0.629970,0.656439,0.829652,0.808376,0.812805,0.558455,0.969703,...,0.427523,0.601445,0.778328,0.714175,0.534086,0.534094,0.666667,0.453607,0.660332,0.005
2886,0.637546,0.687500,0.492318,0.609231,0.698560,0.764508,0.793695,0.808355,0.590457,0.969703,...,0.417512,0.773763,0.737805,0.787221,0.545482,0.545409,0.681506,0.294035,0.631222,0.350


In [11]:
test_data_scaler

,V0,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37
0,0.728470,0.781534,0.511364,0.609231,0.689140,0.851367,0.792150,0.803761,0.711547,0.969703,...,0.279482,0.394955,0.634241,0.738948,0.759926,0.511395,0.482877,0.686495,0.260133,0.606033
1,0.694393,0.797017,0.507843,0.609231,0.654555,0.741670,0.782414,0.795435,0.658065,0.969703,...,0.280326,0.394955,0.704141,0.766641,0.714020,0.463040,0.461562,0.711186,0.294927,0.563198
2,0.645756,0.718750,0.497599,0.624045,0.630736,0.840509,0.782105,0.795435,0.643650,0.969703,...,0.278216,0.335558,0.640356,0.769563,0.726892,0.460166,0.460147,0.711186,0.379939,0.633333
3,0.687268,0.769318,0.505922,0.624825,0.613511,0.872332,0.780405,0.797014,0.677815,0.969703,...,0.278497,0.335558,0.766259,0.761687,0.790167,0.454518,0.454490,0.711186,0.247515,0.606486
4,0.717937,0.788352,0.580666,0.624825,0.633697,0.860352,0.770206,0.796009,0.698717,0.969703,...,0.278216,0.285104,0.666759,0.791286,0.656638,0.454518,0.454490,0.711186,0.314555,0.472549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1920,0.460502,0.506960,0.051857,0.547638,0.689409,1.331711,-0.042961,-0.040626,0.089088,0.636370,...,0.416549,0.411506,0.002640,0.008384,0.000465,0.340864,0.397717,0.444444,0.005608,0.637557
1921,0.253563,0.237216,-0.032010,0.450647,0.451756,1.332834,-0.054242,-0.007895,0.043102,0.787853,...,0.279904,0.561799,0.541134,-0.233613,0.524038,0.340864,0.397717,0.444444,0.005608,0.730769
1922,0.266419,0.221307,0.002881,0.551068,0.466963,1.236615,0.054551,0.105225,-0.072654,0.666667,...,0.279764,0.462360,0.643691,-0.032012,0.671216,0.275069,0.327407,0.518519,0.008157,0.790649
1923,0.259294,0.209659,0.023848,0.507875,0.467232,1.234744,0.132901,0.177864,-0.006199,0.696963,...,0.279764,0.431127,0.777237,0.075076,0.734491,0.295482,0.335185,0.518519,0.007520,0.763348


## PCA方法降维

In [14]:
from sklearn.decomposition import PCA   #主成分分析法

#PCA方法降维
#保留16个主成分
pca = PCA(n_components=16)
new_train_pca_16 = pca.fit_transform(train_data_by_scaler.iloc[:,0:-1])
new_test_pca_16 = pca.transform(test_data_by_scaler)
new_train_pca_16 = pd.DataFrame(new_train_pca_16)
new_test_pca_16 = pd.DataFrame(new_test_pca_16)
new_train_pca_16['target'] = train_data_scaler['target']

In [15]:
#采用 pca 保留16维特征的数据
new_train_pca_16 = new_train_pca_16.fillna(0)
train = new_train_pca_16[new_test_pca_16.columns]
target = new_train_pca_16['target']

# 切分数据 训练数据80% 验证数据20%
train_data,test_data,train_target,test_target=train_test_split(train,target,test_size=0.2,random_state=0)

In [16]:
train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.177673,0.051402,-0.042928,0.596860,0.619306,-0.203065,-0.090735,0.270940,-0.172959,-0.289435,-0.467930,-0.197962,-0.112879,0.046537,-0.007136,0.311447
1,0.005699,0.007641,0.014920,0.386126,0.397348,-0.231888,0.127766,-0.161989,0.038464,0.105252,-0.156610,-0.085433,0.141210,0.023412,0.043470,0.176242
2,0.027371,-0.073686,-0.100606,0.260537,0.402716,-0.175567,0.082570,-0.192245,0.115156,0.156189,-0.140781,-0.051967,0.207804,0.002717,0.121696,0.147129
3,0.135186,-0.109919,-0.055314,0.245710,0.405216,-0.263677,0.078688,-0.110854,0.127537,0.149100,-0.170805,-0.064703,0.185821,0.006167,0.032069,0.070241
4,0.139710,-0.010879,-0.227273,0.164370,0.301791,-0.328074,0.076475,0.077866,0.104619,0.100393,-0.109889,-0.059666,0.188658,0.073855,0.121346,0.031654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2883,-0.104217,0.151533,-0.217488,-0.262431,-0.254555,0.087831,-0.196187,-0.040748,-0.076592,-0.077222,0.043782,0.096107,-0.058520,0.012313,0.018248,0.112132
2884,-0.182793,0.124886,-0.262498,-0.222752,-0.189719,-0.014685,-0.181829,0.095214,-0.116519,-0.142594,0.203034,0.195807,-0.120294,0.072388,0.087020,0.061436
2885,0.083377,0.211972,-0.201123,-0.252699,-0.413525,0.039931,-0.142431,-0.121583,-0.040585,-0.054668,-0.053861,0.138138,-0.108745,0.025744,0.000552,0.065017
2886,0.027927,0.235090,-0.127063,-0.174139,-0.428295,-0.110481,-0.193225,0.064809,-0.011198,-0.111176,0.006924,0.107738,-0.122311,0.075883,0.036571,-0.032719


In [17]:
# 欠拟合

clf = SGDRegressor(max_iter=500, tol=1e-2) 
clf.fit(train_data, train_target)
score_train = mean_squared_error(train_target, clf.predict(train_data))
score_test = mean_squared_error(test_target, clf.predict(test_data))
print("SGDRegressor train MSE:   ", score_train)
print("SGDRegressor test MSE:   ", score_test)

SGDRegressor train MSE:    0.15156831363938905
SGDRegressor test MSE:    0.15589430799272522


In [18]:
# 过拟合

from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(5)
train_data_poly = poly.fit_transform(train_data)
test_data_poly = poly.transform(test_data)
clf = SGDRegressor(max_iter=1000, tol=1e-3) 
clf.fit(train_data_poly, train_target)
score_train = mean_squared_error(train_target, clf.predict(train_data_poly))
score_test = mean_squared_error(test_target, clf.predict(test_data_poly))
print("SGDRegressor train MSE:   ", score_train)
print("SGDRegressor test MSE:   ", score_test)

SGDRegressor train MSE:    0.13344294160101108
SGDRegressor test MSE:    0.14568327954588042


In [19]:
# 正常
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(3)
train_data_poly = poly.fit_transform(train_data)
test_data_poly = poly.transform(test_data)
clf = SGDRegressor(max_iter=1000, tol=1e-3) 
clf.fit(train_data_poly, train_target)
score_train = mean_squared_error(train_target, clf.predict(train_data_poly))
score_test = mean_squared_error(test_target, clf.predict(test_data_poly))
print("SGDRegressor train MSE:   ", score_train)
print("SGDRegressor test MSE:   ", score_test)

SGDRegressor train MSE:    0.13402157172073076
SGDRegressor test MSE:    0.14245010803198943


In [21]:
## L2范数正则化

poly = PolynomialFeatures(3)
train_data_poly = poly.fit_transform(train_data)
test_data_poly = poly.transform(test_data)
clf = SGDRegressor(max_iter=1000, tol=1e-3, penalty= 'L2', alpha=0.0001) # 随机梯度下降
clf.fit(train_data_poly, train_target)
score_train = mean_squared_error(train_target, clf.predict(train_data_poly))
score_test = mean_squared_error(test_target, clf.predict(test_data_poly))
print("SGDRegressor train MSE:   ", score_train)
print("SGDRegressor test MSE:   ", score_test)

SGDRegressor train MSE:    0.13409525189734872
SGDRegressor test MSE:    0.1424272457576181


In [24]:
## L1
poly = PolynomialFeatures(3)
train_data_poly = poly.fit_transform(train_data)
test_data_poly = poly.transform(test_data)
clf = SGDRegressor(max_iter=1000, tol=1e-3, penalty= 'L1', alpha=0.00001) 
clf.fit(train_data_poly, train_target)
score_train = mean_squared_error(train_target, clf.predict(train_data_poly))
score_test = mean_squared_error(test_target, clf.predict(test_data_poly))
print("SGDRegressor train MSE:   ", score_train)
print("SGDRegressor test MSE:   ", score_test)

SGDRegressor train MSE:    0.13403702781444599
SGDRegressor test MSE:    0.14255755106066892


### ElasticNet L1和L2范数加权正则化

In [32]:
poly = PolynomialFeatures(3)
train_data_poly = poly.fit_transform(train_data)
test_data_poly = poly.transform(test_data)
clf = SGDRegressor(max_iter=1000, tol=1e-3, penalty= 'elasticnet', l1_ratio=0.9, alpha=0.00001) 
clf.fit(train_data_poly, train_target)
score_train = mean_squared_error(train_target, clf.predict(train_data_poly))
score_test = mean_squared_error(test_target, clf.predict(test_data_poly))
print("SGDRegressor train MSE:   ", score_train)
print("SGDRegressor test MSE:   ", score_test)

SGDRegressor train MSE:    0.13399977070407354
SGDRegressor test MSE:    0.14256521892697907


## 5.3.3 模型交叉验证¶

### 简单交叉验证

In [30]:
from sklearn.model_selection import train_test_split # 切分数据
# 切分数据 训练数据80% 验证数据20%
train_data,test_data,train_target,test_target=train_test_split(train,target,test_size=0.2,random_state=0)

clf = SGDRegressor(max_iter=1000, tol=1e-3) 
clf.fit(train_data, train_target)
score_train = mean_squared_error(train_target, clf.predict(train_data))
score_test = mean_squared_error(test_target, clf.predict(test_data))
print("SGDRegressor train MSE:   ", score_train)
print("SGDRegressor test MSE:   ", score_test)

SGDRegressor train MSE:    0.1414294479808498
SGDRegressor test MSE:    0.14689202648881405


### 5折交叉验证

In [29]:

from sklearn.model_selection import KFold

kf = KFold(n_splits=5)
for k, (train_index, test_index) in enumerate(kf.split(train)):
    train_data,test_data,train_target,test_target = train.values[train_index],train.values[test_index],target[train_index],target[test_index]
    clf = SGDRegressor(max_iter=1000, tol=1e-3) 
    clf.fit(train_data, train_target)
    score_train = mean_squared_error(train_target, clf.predict(train_data))
    score_test = mean_squared_error(test_target, clf.predict(test_data))
    print(k, " 折", "SGDRegressor train MSE:   ", score_train)
    print(k, " 折", "SGDRegressor test MSE:   ", score_test, '\n')  

0  折 SGDRegressor train MSE:    0.14994315795191443
0  折 SGDRegressor test MSE:    0.10657059837917632 

1  折 SGDRegressor train MSE:    0.13345993047785237
1  折 SGDRegressor test MSE:    0.18200534179263242 

2  折 SGDRegressor train MSE:    0.14702328302029988
2  折 SGDRegressor test MSE:    0.1330497300815027 

3  折 SGDRegressor train MSE:    0.14080142143162613
3  折 SGDRegressor test MSE:    0.16299770412302217 

4  折 SGDRegressor train MSE:    0.1387781726638474
4  折 SGDRegressor test MSE:    0.16583922830880943 



### 留一法 LOO CV

In [28]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
num = 100
for k, (train_index, test_index) in enumerate(loo.split(train)):
    train_data,test_data,train_target,test_target = train.values[train_index],train.values[test_index],target[train_index],target[test_index]
    clf = SGDRegressor(max_iter=1000, tol=1e-3) 
    clf.fit(train_data, train_target)
    score_train = mean_squared_error(train_target, clf.predict(train_data))
    score_test = mean_squared_error(test_target, clf.predict(test_data))
    print(k, " 个", "SGDRegressor train MSE:   ", score_train)
    print(k, " 个", "SGDRegressor test MSE:   ", score_test, '\n') 
    if k >= 9:
        break

0  个 SGDRegressor train MSE:    0.1416001475821287
0  个 SGDRegressor test MSE:    0.01177596667089989 

1  个 SGDRegressor train MSE:    0.14167571241876378
1  个 SGDRegressor test MSE:    0.1266971002181522 

2  个 SGDRegressor train MSE:    0.14162695536484404
2  个 SGDRegressor test MSE:    0.040696797456864074 

3  个 SGDRegressor train MSE:    0.14157969242965912
3  个 SGDRegressor test MSE:    0.0028854069554548572 

4  个 SGDRegressor train MSE:    0.14157055684209924
4  个 SGDRegressor test MSE:    0.011131207290430975 

5  个 SGDRegressor train MSE:    0.14139346827946483
5  个 SGDRegressor test MSE:    0.14201501287350274 

6  个 SGDRegressor train MSE:    0.1415525621205422
6  个 SGDRegressor test MSE:    0.02346414816042782 

7  个 SGDRegressor train MSE:    0.14155476400813374
7  个 SGDRegressor test MSE:    0.0004107062517471701 

8  个 SGDRegressor train MSE:    0.1416190365436507
8  个 SGDRegressor test MSE:    0.09140555963834716 

9  个 SGDRegressor train MSE:    0.14159889153825056
9

### 留P法 LPO CV

In [34]:
from sklearn.model_selection import LeavePOut
lpo = LeavePOut(p=10)
num = 100
for k, (train_index, test_index) in enumerate(lpo.split(train)):
    train_data,test_data,train_target,test_target = train.values[train_index],train.values[test_index],target[train_index],target[test_index]
    clf = SGDRegressor(max_iter=1000, tol=1e-3) 
    clf.fit(train_data, train_target)
    score_train = mean_squared_error(train_target, clf.predict(train_data))
    score_test = mean_squared_error(test_target, clf.predict(test_data))
    print(k, " 10个", "SGDRegressor train MSE:   ", score_train)
    print(k, " 10个", "SGDRegressor test MSE:   ", score_test, '\n') 
    if k >= 9:
        break

0  10个 SGDRegressor train MSE:    0.1419467698950565
0  10个 SGDRegressor test MSE:    0.049216382383040935 

1  10个 SGDRegressor train MSE:    0.1419974402786753
1  10个 SGDRegressor test MSE:    0.044916632218638536 

2  10个 SGDRegressor train MSE:    0.1420174951168486
2  10个 SGDRegressor test MSE:    0.04657378533773585 

3  10个 SGDRegressor train MSE:    0.14195297551879318
3  10个 SGDRegressor test MSE:    0.054161685560437524 

4  10个 SGDRegressor train MSE:    0.14189205953551476
4  10个 SGDRegressor test MSE:    0.06936160133927119 

5  10个 SGDRegressor train MSE:    0.1419902983896831
5  10个 SGDRegressor test MSE:    0.04502278864626115 

6  10个 SGDRegressor train MSE:    0.14190964976727535
6  10个 SGDRegressor test MSE:    0.04931251023927125 

7  10个 SGDRegressor train MSE:    0.14189177452656876
7  10个 SGDRegressor test MSE:    0.05370952382689613 

8  10个 SGDRegressor train MSE:    0.1420265056932699
8  10个 SGDRegressor test MSE:    0.04751919652646081 

9  10个 SGDRegressor t

## 5.3.4 模型超参空间及调参¶

### 穷举网格搜索

In [35]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split # 切分数据
# 切分数据 训练数据80% 验证数据20%
train_data,test_data,train_target,test_target=train_test_split(train,target,test_size=0.2,random_state=0)

randomForestRegressor = RandomForestRegressor()
parameters = {
              'n_estimators':[50, 100, 200],
              'max_depth':[1, 2, 3]
        }


clf = GridSearchCV(randomForestRegressor, parameters, cv=5)
clf.fit(train_data, train_target)

score_test = mean_squared_error(test_target, clf.predict(test_data))

print("RandomForestRegressor GridSearchCV test MSE:   ", score_test)
sorted(clf.cv_results_.keys())

RandomForestRegressor GridSearchCV test MSE:    0.2553954098692129


['mean_fit_time',
 'mean_score_time',
 'mean_test_score',
 'param_max_depth',
 'param_n_estimators',
 'params',
 'rank_test_score',
 'split0_test_score',
 'split1_test_score',
 'split2_test_score',
 'split3_test_score',
 'split4_test_score',
 'std_fit_time',
 'std_score_time',
 'std_test_score']

### 随机参数优化

In [37]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split # 切分数据
# 切分数据 训练数据80% 验证数据20%
train_data,test_data,train_target,test_target=train_test_split(train,target,test_size=0.2,random_state=0)

randomForestRegressor = RandomForestRegressor()
parameters = {
              'n_estimators':[50, 100, 200, 300],
              'max_depth':[1, 2, 3, 4, 5]
        }


clf = RandomizedSearchCV(randomForestRegressor, parameters, cv=5)
clf.fit(train_data, train_target)

score_test = mean_squared_error(test_target, clf.predict(test_data))

print("RandomForestRegressor RandomizedSearchCV test MSE:   ", score_test)
sorted(clf.cv_results_.keys())

RandomForestRegressor RandomizedSearchCV test MSE:    0.19652311310936638


['mean_fit_time',
 'mean_score_time',
 'mean_test_score',
 'param_max_depth',
 'param_n_estimators',
 'params',
 'rank_test_score',
 'split0_test_score',
 'split1_test_score',
 'split2_test_score',
 'split3_test_score',
 'split4_test_score',
 'std_fit_time',
 'std_score_time',
 'std_test_score']

### Lgb 调参

In [38]:
clf = lgb.LGBMRegressor(num_leaves=31)

parameters = {
    'learning_rate': [0.01, 0.1, 1],
    'n_estimators': [20, 40]
}

clf = GridSearchCV(clf, parameters, cv=5)
clf.fit(train_data, train_target)

print('Best parameters found by grid search are:', clf.best_params_)
score_test = mean_squared_error(test_target, clf.predict(test_data))
print("LGBMRegressor RandomizedSearchCV test MSE:   ", score_test)

Best parameters found by grid search are: {'learning_rate': 0.1, 'n_estimators': 40}
LGBMRegressor RandomizedSearchCV test MSE:    0.15208604025135006


### Lgb 线下验证

In [39]:
train_data2 = pd.read_csv('./zhengqi_train.txt',sep='\t')
test_data2 = pd.read_csv('./zhengqi_test.txt',sep='\t')

train_data2_f = train_data2[test_data2.columns].values
train_data2_target = train_data2['target'].values

In [ ]:
# lgb 模型
from sklearn.model_selection  import KFold
import lightgbm as lgb
import numpy as np


# 5折交叉验证
Folds=5
kf = KFold(len(train_data2_f), n_splits=Folds, random_state=100, shuffle=True)
# 记录训练和预测MSE
MSE_DICT = {
    'train_mse':[],
    'test_mse':[]
}

# 线下训练预测
for i, (train_index, test_index) in enumerate(kf.split(train_data2_f)):
    # lgb树模型
    lgb_reg = lgb.LGBMRegressor(
        learning_rate=0.01,
        max_depth=-1,
        n_estimators=100,
        boosting_type='gbdt',
        random_state=100,
        objective='regression',
    )
   
    # 切分训练集和预测集
    X_train_KFold, X_test_KFold = train_data2_f[train_index], train_data2_f[test_index]
    y_train_KFold, y_test_KFold = train_data2_target[train_index], train_data2_target[test_index]
    
    # 训练模型
#     reg.fit(X_train_KFold, y_train_KFold)
    lgb_reg.fit(
            X=X_train_KFold,y=y_train_KFold,
            eval_set=[(X_train_KFold, y_train_KFold),(X_test_KFold, y_test_KFold)],
            eval_names=['Train','Test'],
            early_stopping_rounds=100,
            eval_metric='MSE',
            verbose=50
        )


    # 训练集预测 测试集预测
    y_train_KFold_predict = lgb_reg.predict(X_train_KFold,num_iteration=lgb_reg.best_iteration_)
    y_test_KFold_predict = lgb_reg.predict(X_test_KFold,num_iteration=lgb_reg.best_iteration_) 
    
#     print('第{}折 训练和预测 训练MSE 预测MSE'.format(i))
    train_mse = mean_squared_error(y_train_KFold_predict, y_train_KFold)
#     print('------\n', '训练MSE\n', train_mse, '\n------')
    test_mse = mean_squared_error(y_test_KFold_predict, y_test_KFold)
#     print('------\n', '预测MSE\n', test_mse, '\n------\n')
    
    MSE_DICT['train_mse'].append(train_mse)
    MSE_DICT['test_mse'].append(test_mse)
print('------\n', '训练MSE\n', MSE_DICT['train_mse'], '\n', np.mean(MSE_DICT['train_mse']), '\n------')
print('------\n', '预测MSE\n', MSE_DICT['test_mse'], '\n', np.mean(MSE_DICT['test_mse']), '\n------')

Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.433641	Test's l2: 1.26094
[100]	Train's l2: 0.225389	Test's l2: 0.395946
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.225389	Test's l2: 0.395946
Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.434194	Test's l2: 0.21387
[100]	Train's l2: 0.225623	Test's l2: 0.142998
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.225623	Test's l2: 0.142998
Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.43384	Test's l2: 1.22148
[100]	Train's l2: 0.225411	Test's l2: 0.903757
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.225411	Test's l2: 0.903757
Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.434124	Test's l2: 0.311844
[100]	Train's l2: 0.22575	Test's l2: 0.0761824
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.22575	Test's l2: 0.0761824
Trai

Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.434211	Test's l2: 0.107417
[100]	Train's l2: 0.225736	Test's l2: 0.0865263
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.225736	Test's l2: 0.0865263
Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.434102	Test's l2: 0.17063
[100]	Train's l2: 0.225641	Test's l2: 0.0210496
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.225641	Test's l2: 0.0210496
Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.434186	Test's l2: 0.108701
[100]	Train's l2: 0.225818	Test's l2: 0.0717651
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.225818	Test's l2: 0.0717651
Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.43412	Test's l2: 0.192861
[100]	Train's l2: 0.225641	Test's l2: 0.102299
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.225641	Test's l2: 0.102

[100]	Train's l2: 0.225655	Test's l2: 0.209585
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.225655	Test's l2: 0.209585
Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.43421	Test's l2: 0.0328662
[100]	Train's l2: 0.225719	Test's l2: 0.0132314
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.225719	Test's l2: 0.0132314
Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.433749	Test's l2: 1.44239
[100]	Train's l2: 0.225468	Test's l2: 0.73053
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.225468	Test's l2: 0.73053
Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.434186	Test's l2: 0.00615191
[100]	Train's l2: 0.225751	Test's l2: 0.00787524
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.225751	Test's l2: 0.00787524
Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.434229	Test's l2: 0.04

Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.433355	Test's l2: 3.38676
[100]	Train's l2: 0.225407	Test's l2: 2.04894
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.225407	Test's l2: 2.04894
Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.434198	Test's l2: 0.0188443
[100]	Train's l2: 0.225712	Test's l2: 0.0151944
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.225712	Test's l2: 0.0151944
Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.43397	Test's l2: 1.03573
[100]	Train's l2: 0.22553	Test's l2: 0.783529
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.22553	Test's l2: 0.783529
Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.434209	Test's l2: 0.00847547
[100]	Train's l2: 0.225764	Test's l2: 0.00205392
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.225764	Test's l2: 0.0020539

[50]	Train's l2: 0.434198	Test's l2: 0.0356108
[100]	Train's l2: 0.225668	Test's l2: 0.00300899
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.225668	Test's l2: 0.00300899
Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.434208	Test's l2: 3.92946e-05
[100]	Train's l2: 0.225684	Test's l2: 0.0164476
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.225684	Test's l2: 0.0164476
Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.434024	Test's l2: 0.292803
[100]	Train's l2: 0.22566	Test's l2: 0.212196
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.22566	Test's l2: 0.212196
Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.433863	Test's l2: 0.965786
[100]	Train's l2: 0.225515	Test's l2: 0.599638
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.225515	Test's l2: 0.599638
Training until validation scores don't improve for 100

...

[50]	Train's l2: 0.434122	Test's l2: 0.230579
[100]	Train's l2: 0.225702	Test's l2: 0.14645
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.225702	Test's l2: 0.14645
Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.434185	Test's l2: 0.0304434
[100]	Train's l2: 0.225733	Test's l2: 0.00734217
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.225733	Test's l2: 0.00734217
Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.434024	Test's l2: 0.710854
[100]	Train's l2: 0.225694	Test's l2: 0.355256
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.225694	Test's l2: 0.355256
Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.434035	Test's l2: 0.607289
[100]	Train's l2: 0.22564	Test's l2: 0.306935
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.22564	Test's l2: 0.306935
Training until validation scores don't improve for 100 rounds

[50]	Train's l2: 0.434044	Test's l2: 0.700189
[100]	Train's l2: 0.22565	Test's l2: 0.379595
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.22565	Test's l2: 0.379595
Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.434235	Test's l2: 0.126777
[100]	Train's l2: 0.225713	Test's l2: 0.0642162
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.225713	Test's l2: 0.0642162
Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.434122	Test's l2: 0.15751
[100]	Train's l2: 0.225688	Test's l2: 0.168706
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.225688	Test's l2: 0.168706
Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.43395	Test's l2: 0.750991
[100]	Train's l2: 0.225543	Test's l2: 0.408077
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.225543	Test's l2: 0.408077
Training until validation scores don't improve for 100 rounds
[5

[100]	Train's l2: 0.225709	Test's l2: 0.00105866
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.225709	Test's l2: 0.00105866
Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.433993	Test's l2: 0.650071
[100]	Train's l2: 0.225569	Test's l2: 0.45236
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.225569	Test's l2: 0.45236
Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.434205	Test's l2: 4.05581e-05
[100]	Train's l2: 0.225666	Test's l2: 0.0348257
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.225666	Test's l2: 0.0348257
Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.434239	Test's l2: 0.0840817
[100]	Train's l2: 0.225714	Test's l2: 0.0116107
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.225714	Test's l2: 0.0116107
Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.434155	Test's l2:

[100]	Train's l2: 0.225767	Test's l2: 0.000863513
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.225767	Test's l2: 0.000863513
Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.434118	Test's l2: 0.404045
[100]	Train's l2: 0.225662	Test's l2: 0.272073
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.225662	Test's l2: 0.272073
Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.434237	Test's l2: 0.00131292
[100]	Train's l2: 0.225744	Test's l2: 0.00872275
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.225744	Test's l2: 0.00872275
Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.434114	Test's l2: 0.390943
[100]	Train's l2: 0.22564	Test's l2: 0.206385
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.22564	Test's l2: 0.206385
Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.434103	Test's l2:

Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.434205	Test's l2: 0.082718
[100]	Train's l2: 0.225615	Test's l2: 0.0376504
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.225615	Test's l2: 0.0376504
Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.434268	Test's l2: 0.0028162
[100]	Train's l2: 0.225724	Test's l2: 0.000913098
Did not meet early stopping. Best iteration is:
[100]	Train's l2: 0.225724	Test's l2: 0.000913098
Training until validation scores don't improve for 100 rounds


In [ ]:
print(__doc__)
import numpy as np
import matplotlib.pyplot as plt
from sklearn import model_selection 
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import learning_curve


def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt


X = train_data2[test_data2.columns].values
y = train_data2['target'].values


title = "LinearRegression"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = model_selection.ShuffleSplit(X.shape[0], n_splits=100,
                                   test_size=0.2, random_state=0)

estimator = SGDRegressor()
plot_learning_curve(estimator, title, X, y, ylim=(0.7, 1.01), cv=cv, n_jobs=-1)


### 验证曲线

In [ ]:
print(__doc__)

import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import validation_curve

X = train_data2[test_data2.columns].values
y = train_data2['target'].values
# max_iter=1000, tol=1e-3, penalty= 'L1', alpha=0.00001

param_range = [0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001]
train_scores, test_scores = validation_curve(
    SGDRegressor(max_iter=1000, tol=1e-3, penalty= 'L1'), X, y, param_name="alpha", param_range=param_range,
    cv=10, scoring='r2', n_jobs=1)
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

plt.title("Validation Curve with SGDRegressor")
plt.xlabel("alpha")
plt.ylabel("Score")
plt.ylim(0.0, 1.1)
plt.semilogx(param_range, train_scores_mean, label="Training score", color="r")
plt.fill_between(param_range, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.2, color="r")
plt.semilogx(param_range, test_scores_mean, label="Cross-validation score",
             color="g")
plt.fill_between(param_range, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.2, color="g")
plt.legend(loc="best")
plt.show()